In [5]:
import h5py
import numpy as np

In [6]:
actions = []
dones = []
obs = []
rewards = []
robot_states = []
states = []

with h5py.File("/home/fri-skills/LIBERO/libero/datasets/libero_90/KITCHEN_SCENE2_put_the_black_bowl_at_the_front_on_the_plate_demo.hdf5", "r") as file:
    data_group = file['data']
    print(file.keys())
    print(data_group.keys())
    for sample in data_group.keys():
        print(sample)
        curr_sample = data_group[sample]
        # print(curr_sample.keys())
        curr_sample["actions"][:]
        actions.append(curr_sample["actions"][:])
        actions_np = np.array(actions)
        # print(curr_sample["actions"])
        # print(actions_np.shape)
        dones.append(curr_sample["dones"][:])
        obs_keys = curr_sample["obs"].keys()
        print(obs_keys)
        
        # print(obs_keys)
        # obs_agentviews = curr_sample["obs"]["agentview_rgb"]
        # print(obs_agentviews)
        # obs_eeori = curr_sample["obs"]["ee_ori"]
        # print(obs_eeori)
        # print(curr_sample["obs"]["ee_states"])
        for key in obs_keys:
            print(curr_sample["obs"][key])
        # for obk in obs_keys:
        #     print(obk)
        # break

<KeysViewHDF5 ['data']>
<KeysViewHDF5 ['demo_0', 'demo_1', 'demo_10', 'demo_11', 'demo_12', 'demo_13', 'demo_14', 'demo_15', 'demo_16', 'demo_17', 'demo_18', 'demo_19', 'demo_2', 'demo_20', 'demo_21', 'demo_22', 'demo_23', 'demo_24', 'demo_25', 'demo_26', 'demo_27', 'demo_28', 'demo_29', 'demo_3', 'demo_30', 'demo_31', 'demo_32', 'demo_33', 'demo_34', 'demo_35', 'demo_36', 'demo_37', 'demo_38', 'demo_39', 'demo_4', 'demo_40', 'demo_41', 'demo_42', 'demo_43', 'demo_44', 'demo_45', 'demo_46', 'demo_47', 'demo_48', 'demo_49', 'demo_5', 'demo_6', 'demo_7', 'demo_8', 'demo_9']>
demo_0
<KeysViewHDF5 ['agentview_rgb', 'ee_ori', 'ee_pos', 'ee_states', 'eye_in_hand_rgb', 'gripper_states', 'joint_states']>
<HDF5 dataset "agentview_rgb": shape (134, 128, 128, 3), type "|u1">
<HDF5 dataset "ee_ori": shape (134, 3), type "<f8">
<HDF5 dataset "ee_pos": shape (134, 3), type "<f8">
<HDF5 dataset "ee_states": shape (134, 6), type "<f8">
<HDF5 dataset "eye_in_hand_rgb": shape (134, 128, 128, 3), type "|

/tmp/ipykernel_53769/1535777110.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  actions_np = np.array(actions)


In [7]:
import torch.nn as nn
import torch

encoder_input_layer = nn.Linear(7, 128)
array = np.random.rand(21, 22, 7)
print(array.shape)

tensor_input = torch.tensor(array, dtype=torch.float32)  # Specify dtype
print(tensor_input.shape)

# Pass the tensor to the linear layer
result = encoder_input_layer(tensor_input)
print(result.shape)

(21, 22, 7)
torch.Size([21, 22, 7])
torch.Size([21, 22, 128])


In [8]:
from robomimic.utils.obs_utils import OBS_MODALITY_CLASSES

print(OBS_MODALITY_CLASSES.keys())  # List all recognized observation modalities


dict_keys(['rgb', 'depth', 'scan', 'low_dim'])


In [13]:
from Models.PolicyNetwork import PolicyNetwork
from Models.CriticNetwork import CriticNetwork
from Models.ReplayBuffer import ReplayBuffer
from Models.SkillEmbeddingPrior import SkillEmbeddingAndPrior
from Models.SkillPriorNet import SkillPriorNet
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from libero.lifelong.datasets import get_dataset
from torch.distributions import Normal
from torch.distributions.kl import kl_divergence
from colorama import Fore, Style
import matplotlib.pyplot as plt
import numpy as np
import gym
from libero.libero import benchmark
from libero.libero.envs import OffScreenRenderEnv
import os
from libero_env import make_env, CFG


from libero.libero import benchmark, get_libero_path
from libero.libero.envs import OffScreenRenderEnv

use_arnav_code = False
if use_arnav_code:
    env = make_env(CFG("libero_object", 0, "rgb"))
else:
    benchmark_dict = benchmark.get_benchmark_dict()
    task_suite_name = "libero_10" # can also choose libero_spatial, libero_object, etc.
    task_suite = benchmark_dict[task_suite_name]()

    # retrieve a specific task
    task_id = 0
    task = task_suite.get_task(task_id)
    task_name = task.name
    task_description = task.language
    task_bddl_file = os.path.join(get_libero_path("bddl_files"), task.problem_folder, task.bddl_file)
    print(task_bddl_file)
    print(f"[info] retrieving task {task_id} from suite {task_suite_name}, the " + \
        f"language instruction is {task_description}, and the bddl file is {task_bddl_file}")

    # step over the environment
    env_args = {
        "bddl_file_name": task_bddl_file,
        "camera_heights": 128,
        "camera_widths": 128
    }
    env = OffScreenRenderEnv(**env_args)

    print(env.env.observation_spec().keys())
    env.seed(0)
    env.reset()
    init_states = task_suite.get_task_init_states(task_id) # for benchmarking purpose, we fix the a set of initial states
    init_state_id = 0
    env.set_init_state(init_states[init_state_id])
    # print(init_states[0].shape);quit()
    dummy_action = [0.] * 7

"""
doesnt work b/c observation() never gets called
class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env: gym.Env):
        super().__init__(env)
        self.observation_space = spaces.flatten_space(env.env.observation_spec())
        self.state_dim = len(self.observation_space)

    def observation(self, observation):
        # return spaces.flatten(self.env.env.observation_spec(), observation)
        state_size = 0
        state_dim = 0
        state_col_arr = []
        for entry in list(observation.values()):
            # state_size += 1
            try:
                for entry2 in entry:
                    try:
                        for entry3 in entry2:
                            state_size += 1
                            state_col_arr.append(entry3)
                    except:
                        state_size += 1
                        state_col_arr.append(entry2)
            except:
                state_size += 1
                state_col_arr.append(entry)
            # state_dim = state_size
            # print(state_size)
            # state_size = 0
        state_dim = state_size
        raise ''
        print('START',np.array(state_col_arr),'END');quit()
        return np.array(state_col_arr)
"""
# env.observation_space OrderedDict space
# from gym.wrappers import FlattenObservation
# env = FlattenObservation(env)
# Box
for step in range(10):
    # obs OrderedDict
    obs, reward, done, info = env.step(dummy_action)
    print(obs.keys())
    break
    # obs vector
    # print(len(list(obs.values())));quit()

# state_dim = env.observation_space.shape[0]
# action_dim = list(env.env.action_spec)[0].shape[0]



[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
/home/fri-skills/LIBERO/libero/libero/./bddl_files/libero_10/LIVING_ROOM_SCENE2_put_both_the_alphabet_soup_and_the_tomato_sauce_in_the_basket.bddl
[info] retrieving task 0 from suite libero_10, the language instruction is put both the alphabet soup and the tomato sauce in the basket, and the bddl file is /home/fri-skills/LIBERO/libero/libero/./bddl_files/libero_10/LIVING_ROOM_SCENE2_put_both_the_alphabet_soup_and_the_tomato_sauce_in_the_basket.bddl
odict_keys(['robot0_joint_pos', 'robot0_joint_pos_cos', 'robot0_joint_pos_sin', 'robot0_joint_vel', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_gripper_qpos', 'robot0_gripper_qvel', 'agentview_image', 'robot0_eye_in_hand_image', 'alphabet_soup_1_pos', 'alphabet_soup_1_quat', 'alphabet_soup_1_to_robot0_eef_pos', 'alphabet_soup_1_to_robot0_eef_quat', 'cream_cheese_1_pos', 'cream_cheese_1_quat', 'cream_cheese_1_to_robot0_eef_pos', 'cream_cheese_1_to_robot0_eef_quat', 'tomato_sauce_1_pos',